# Spotify Tracks Model Building Notebook

In [ ]:
def build_model(embedding_size=50, classification=False, track_index=None, album_index=None):
    from keras.layers import Input, Embedding, Dot, Reshape, Dense
    from keras.models import Model
    
    # Validate the indices
    if track_index is None or album_index is None:
        raise ValueError("track_index and album_index must be provided")
    
    # Both inputs are 1-dimensional
    track_input = Input(name='track', shape=[1])
    album_input = Input(name='album', shape=[1])
    
    # Embedding the track (shape will be (None, 1, 50))
    track_embedding = Embedding(name='track_embedding',
                                input_dim=len(track_index),
                                output_dim=embedding_size)(track_input)
    
    # Embedding the album (shape will be (None, 1, 50))
    album_embedding = Embedding(name='album_embedding',
                                input_dim=len(album_index),
                                output_dim=embedding_size)(album_input)
    
    # Merge the layers with a dot product along the second axis (shape will be (None, 1, 1))
    merged = Dot(name='dot_product', normalize=True, axes=2)([track_embedding, album_embedding])
    
    # Reshape to be a single number (shape will be (None, 1))
    merged = Reshape(target_shape=[1])(merged)
    
    # If classification, add an extra layer and loss function is binary cross entropy
    if classification:
        merged = Dense(1, activation='sigmoid')(merged)
        model = Model(inputs=[track_input, album_input], outputs=merged)
        model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
    # Otherwise, loss function is mean squared error
    else:
        model = Model(inputs=[track_input, album_input], outputs=merged)
        model.compile(optimizer='Adam', loss='mse')
    
    return model